 **Workspace Setup**

 For this summarization model, you must adjust your runtime space from CPU to GPU. We do this for the express purpose of reducing total runtime for the summarization model [Adjust this explaination]

To switch from CPU to GPU, select:
Runtime > Change runtime type > T4 GPU



**Necessary User Imports and Downloads**

In [1]:
!pip install PyMuPDF -q
!pip install -U datasets transformers ninja -q
!pip install -U sentencepiece -q
!pip install clean-text[gpl] -q
!pip install ads -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import time
import multiprocessing
import re
import requests
import os
import concurrent.futures
import fitz

#For summarization
from transformers import pipeline
import torch
from cleantext import clean
from pathlib import Path
import logging
from datasets import load_dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

#From Expertise Finder v2.0
import TextAnalysis as TA

#For N-grams
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Citing this code**

This code is an enhancement or addition to the Expertise finding tool developed by Volz et al. 2023. When citing this particular summarization code, please use Hoare et al. 2024.

Import .csv file produced from Expertise Finder v2.0 code (*Helfenbein et al. 2023)*. This code will only work with results produced by ADS_search.

In [3]:
datf = pd.read_csv('Insert .csv file here')
datf.set_index(datf.columns[0], inplace=True, drop=True)

datf

,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type
Input Author,,,,,,,,,,,,,
"Abdullah, Ajamu",Howard University,"Abdullah, Ajamu",2014AAS...22324403A,New Star Formation in NGC 3690,2014-01-00,NaN,Howard University,NGC 3690 is a system of merging spiral galaxie...,2014AAS...22324403A,"[('star', 5), ('system', 3), ('merging', 3), (...","[(('star', 'formation'), 3), (('ngc', 'system'...","[(('ngc', 'system', 'merging'), 1), (('system'...",Clean
"Allen, Walter W., Jr.",Howard University,"Allen, Walter W., Jr.",2009PhDT.........1A,The canonical proper-time theory applied to pa...,2009-00-00,"Canonical proper-time, Euler-Lagrange equation...",Howard University,This research dissertation has three objective...,2009PhDT.........1A,"[('equation', 6), ('formulation', 5), ('proper...","[(('proper', 'time'), 5), (('canonical', 'prop...","[(('canonical', 'proper', 'time'), 4), (('rese...",Dirty
"Anusie, Steve",Howard University,"Anusie, Steve",2019AAS...23325917A,The Mini-MUSCLES Treasury Survey: Measurements...,2019-01-00,NaN,"Howard University, Washington,\nDC, United States",Astronomers are becoming more interested in fi...,2019AAS...23325917A,"[('star', 9), ('flare', 5), ('observation', 5)...","[(('light', 'curve'), 4), (('dwarf', 'star'), ...","[(('muscles', 'treasury', 'survey'), 2), (('as...",Clean
"Bacha, Tulu",Howard University,"Bacha, Tulu",2011PhDT.........7B,Effects of Cloud on Goddard Lidar Observatory ...,2011-00-00,"Earth sciences, Clouds, Errors, Wind speed, Op...",Howard University,"The Goddard Lidar Observatory for Wind (GLOW),...",2011PhDT.........7B,"[('wind', 10), ('error', 10), ('glow', 9), ('c...","[(('wind', 'speed'), 4), (('atmospheric', 'var...","[(('goddard', 'lidar', 'observatory'), 2), (('...",Dirty
"Blake, Ameer",Howard University,"Blake, Ameer",2016AAS...22713710B,A CubeSat to Search for Transiting Planets Aro...,2016-01-00,NaN,Howard University,The goal of this project is to further our gro...,2016AAS...22713710B,"[('star', 6), ('cubesat', 6), ('disk', 4), ('p...","[(('debris', 'disk'), 2), (('beta', 'pic'), 2)...","[(('goal', 'project', 'growing'), 1), (('proje...",Clean
"Brown, Beth A.","Howard University, Howard University","Brown, Beth A., Brown, Beth A.","2010AIPC.1280....3B,\n2008CQGra..25j5026B","Causal Structures of Dynamic Black Holes, Cons...","2010-10-00, 2008-05-00","black holes, space-time configurations, satell...","NASA Goddard Space Flight\nCenter, Greenbelt, ...","Dynamic space-times, especially those manifest...","10.1063/1.3507200, arXiv:0904.4192, 2010AIPC.1...","[('black', 4), ('hole', 4), ('diagram', 3), ('...","[(('black', 'hole'), 4), (('large', 'scale'), ...","[(('large', 'scale', 'causal'), 2), (('black',...","Dirty, Dirty"
"Coley, J. B.",Howard University,"Coley, J. B.",2020AAS...23545701C,A Study of the Extragalactic Gamma-ray Binary ...,2020-01-00,NaN,"Department of Physics, Howard\nUniversity, Was...","We report on XMM-Newton, Nuclear Spectroscopic...",2020AAS...23545701C,"[('ray', 6), ('xmm', 4), ('newton', 4), ('tele...","[(('xmm', 'newton'), 4), (('light', 'curve'), ...","[(('report', 'xmm', 'newton'), 1), (('xmm', 'n...",Clean
"Coley, Joel","Howard University, Howard University","Coley, Joel, Coley, Joel","2023AAS...24142807C,\n2022HEAD...1940601C",A Study of the 2020 Type II Giant Outburst of ...,"2023-01-00, 2022-04-00",",","Howard University, Howard\nUniversity",We report on Neutron Star Interior Composition...,"2023AAS...24142807C, 2022HEAD...1940601C","[('nustar', 10), ('kev', 10), ('luminosity', 1...","[(('nicer', 'nustar'), 6), (('neutron', 'star'...","[(('fe', 'fluorescent', 'emission'), 4), (('re...","Clean, Dirty"
"Coley, Joel B.","Howard University, Howard University, Howard U...","Coley, Joel B., Coley, Joel B., Coley, Joel B.","2019ApJ...879...34C,\n2019AAS...23323601C,\n20...",A Study of the 20 day Superorbital Modulation ...,"2019-07-00, 2019-01-00, 201

**Locate arXiv identifiers for papers**

For proof of functionality, we choose to focus on arXiv published papers, due to the easy to navigate API of arXiv. The identifier is pulled so that the pdf can be easily downloaded. Future work includes expanding on publication websites we pull from.

In [4]:
target = r'arXiv:(\S+)'

def process_row_id(row):
  identifiers = row['Identifier'].split(',')
  arXiv_list = []

  for identifier in identifiers:
    match = re.search(target, identifier)
    if match:
      arXiv = match.group(1)
      arXiv_list.append(arXiv)
    else:
      arXiv_list.append(None)
  if any(arXiv_list):
    return ', '.join(str(x) for x in arXiv_list if x is not None)
  else:
    return 'None'

datf['arxiv-id'] = datf.apply(process_row_id, axis = 1)
datf

,Input Institution,First Author,Bibcode,Title,Publication Date,Keywords,Affiliations,Abstract,Identifier,Top 10 Words,Top 10 Bigrams,Top 10 Trigrams,Data Type,arxiv-id
Input Author,,,,,,,,,,,,,,
"Abdullah, Ajamu",Howard University,"Abdullah, Ajamu",2014AAS...22324403A,New Star Formation in NGC 3690,2014-01-00,NaN,Howard University,NGC 3690 is a system of merging spiral galaxie...,2014AAS...22324403A,"[('star', 5), ('system', 3), ('merging', 3), (...","[(('star', 'formation'), 3), (('ngc', 'system'...","[(('ngc', 'system', 'merging'), 1), (('system'...",Clean,None
"Allen, Walter W., Jr.",Howard University,"Allen, Walter W., Jr.",2009PhDT.........1A,The canonical proper-time theory applied to pa...,2009-00-00,"Canonical proper-time, Euler-Lagrange equation...",Howard University,This research dissertation has three objective...,2009PhDT.........1A,"[('equation', 6), ('formulation', 5), ('proper...","[(('proper', 'time'), 5), (('canonical', 'prop...","[(('canonical', 'proper', 'time'), 4), (('rese...",Dirty,None
"Anusie, Steve",Howard University,"Anusie, Steve",2019AAS...23325917A,The Mini-MUSCLES Treasury Survey: Measurements...,2019-01-00,NaN,"Howard University, Washington,\nDC, United States",Astronomers are becoming more interested in fi...,2019AAS...23325917A,"[('star', 9), ('flare', 5), ('observation', 5)...","[(('light', 'curve'), 4), (('dwarf', 'star'), ...","[(('muscles', 'treasury', 'survey'), 2), (('as...",Clean,None
"Bacha, Tulu",Howard University,"Bacha, Tulu",2011PhDT.........7B,Effects of Cloud on Goddard Lidar Observatory ...,2011-00-00,"Earth sciences, Clouds, Errors, Wind speed, Op...",Howard University,"The Goddard Lidar Observatory for Wind (GLOW),...",2011PhDT.........7B,"[('wind', 10), ('error', 10), ('glow', 9), ('c...","[(('wind', 'speed'), 4), (('atmospheric', 'var...","[(('goddard', 'lidar', 'observatory'), 2), (('...",Dirty,None
"Blake, Ameer",Howard University,"Blake, Ameer",2016AAS...22713710B,A CubeSat to Search for Transiting Planets Aro...,2016-01-00,NaN,Howard University,The goal of this project is to further our gro...,2016AAS...22713710B,"[('star', 6), ('cubesat', 6), ('disk', 4), ('p...","[(('debris', 'disk'), 2), (('beta', 'pic'), 2)...","[(('goal', 'project', 'growing'), 1), (('proje...",Clean,None
"Brown, Beth A.","Howard University, Howard University","Brown, Beth A., Brown, Beth A.","2010AIPC.1280....3B,\n2008CQGra..25j5026B","Causal Structures of Dynamic Black Holes, Cons...","2010-10-00, 2008-05-00","black holes, space-time configurations, satell...","NASA Goddard Space Flight\nCenter, Greenbelt, ...","Dynamic space-times, especially those manifest...","10.1063/1.3507200, arXiv:0904.4192, 2010AIPC.1...","[('black', 4), ('hole', 4), ('diagram', 3), ('...","[(('black', 'hole'), 4), (('large', 'scale'), ...","[(('large', 'scale', 'causal'), 2), (('black',...","Dirty, Dirty","0904.4192, 0710.2032"
"Coley, J. B.",Howard University,"Coley, J. B.",2020AAS...23545701C,A Study of the Extragalactic Gamma-ray Binary ...,2020-01-00,NaN,"Department of Physics, Howard\nUniversity, Was...","We report on XMM-Newton, Nuclear Spectroscopic...",2020AAS...23545701C,"[('ray', 6), ('xmm', 4), ('newton', 4), ('tele...","[(('xmm', 'newton'), 4), (('light', 'curve'), ...","[(('report', 'xmm', 'newton'), 1), (('xmm', 'n...",Clean,None
"Coley, Joel","Howard University, Howard University","Coley, Joel, Coley, Joel","2023AAS...24142807C,\n2022HEAD...1940601C",A Study of the 2020 Type II Giant Outburst of ...,"2023-01-00, 2022-04-00",",","Howard University, Howard\nUniversity",We report on Neutron Star Interior Composition...,"2023AAS...24142807C, 2022HEAD...1940601C","[('nustar', 10), ('kev', 10), ('luminosity', 1...","[(('nicer', 'nustar'), 6), (('neutron', 'star'...","[(('fe', 'fluorescent', 'emission'), 4), (('re...","Clean, Dirty",None
"Coley, Joel B.","Howard University, Howard University, Howard U...","Coley, Joel B., Coley, Joel B., Coley, Joel B.","2019ApJ...879...34C,\n2019AAS...23323601C,\n20...",A Study of

**Download PDF version of ArXiv papers**

Run time for 20 papers is approximately 10s.

In [5]:
download_directory = '/content/'
pdf_filenames = []

def download_pdf(identifier):
    pdf_url = f'https://arxiv.org/pdf/{identifier}.pdf'
    save_path = os.path.join(download_directory, f'{identifier.replace("/", "_")}.pdf')
    response = requests.get(pdf_url)

    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        return save_path
    return None

all_identifiers = [identifier for row in datf['arxiv-id'].str.split(', ') for identifier in row if identifier != 'None']

with concurrent.futures.ThreadPoolExecutor(max_workers = 20) as executor:
    results = list(executor.map(download_pdf, all_identifiers))

pdf_filenames.extend(result for result in results if result is not None)

#print(pdf_filenames)

**Process Text Files**

Large Language Models need information passed through in the form of an ASCII file to perfom its functions properly. Information from the original pdf versions, like tables, references, acknowledgements, etc. can result in hallucinations in the model. Removing this information is vital after converting the infromation from pdf to txt.

In [14]:
def remove_text_in_brackets(text):
  pattern = r'\([^)]*\)|\[[^\]]*\]'
  return re.sub(pattern, '', text)

def remove_lines_starting_with_number_or_symbol(text):
  pattern = r'^[\d\W].*$'
  return re.sub(pattern, '', text, flags=re.MULTILINE)

def remove_lines_with_one_word(text):
  lines = text.split('\n')
  pattern = r'^\s*\w+\s*$'
  filtered_lines = [line for line in lines if not re.match(pattern, line)]
  return '\n'.join(filtered_lines)

def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line for line in lines if line.strip() != '']
  return '\n'.join(non_empty_lines)

def find_and_remove_references(text, keywords):
  earliest_position = -1
  for keyword in keywords:
    position = text.find(keyword)
    if position != -1:
      earliest_position = position if earliest_position == -1 else min(position, earliest_position)
  if earliest_position != -1:
    text = text[:earliest_position]
  return text

def process_text(text):
  keywords = ["REFERENCES", "ACKNOWLEDGEMENTS", "References", "Acknowledgements"]

  text = find_and_remove_references(text, keywords)
  text = find_and_remove_references(text, keywords)
  text = remove_text_in_brackets(text)
  text = remove_lines_starting_with_number_or_symbol(text)
  text = remove_lines_with_one_word(text)
  text = remove_empty_lines(text)

  return text

def extract_and_process_text(pdf_path, output_path):
  pdf_doc = fitz.open(pdf_path)
  text = ''
  for page in  pdf_doc:
    text += page.get_text()

  lines = text.split('\n')
  filtered_lines = [line for line in lines if len(line.strip()) > 1]
  filtered_text = '\n'.join(filtered_lines)
  cleaned_text = process_text(filtered_text)

  with open(output_path, 'w', encoding = 'utf-8') as output_file:
    output_file.write(cleaned_text)
  return output_path

def process_pdf(pdf_filename):
  text_filename = pdf_filename.replace('.pdf','.txt')
  cleaned_text = extract_and_process_text(pdf_filename, text_filename)
  return text_filename

text_filenames = []

num_workers = 100

with concurrent.futures.ThreadPoolExecutor(max_workers = num_workers) as executor:
  results = list(executor.map(process_pdf, pdf_filenames))

text_filenames.extend(result for result in results if result is not None)

#print(text_filenames)

**Summarization Model**

In [15]:
hf_tag = "pszemraj/led-large-book-summary"

_device = 0 if torch.cuda.is_available() else -1

model = AutoModelForSeq2SeqLM.from_pretrained(hf_tag,).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(hf_tag,)

token_batch_length = 3072
batch_stride = 50

session_settings = {}
session_settings['token_batch_length'] = token_batch_length
session_settings['batch_stride'] = batch_stride

number_beams = 12
min_length =  5
max_len_ratio = 8
length_penalty =  0.5

settings = {
    'min_length':min_length,
    'max_length':int(token_batch_length//max_len_ratio),
    'no_repeat_ngram_size':7,
    'encoder_no_repeat_ngram_size' :7,
    'repetition_penalty':3.7,
    'num_beams':number_beams,
    'length_penalty':length_penalty,
    'early_stopping':True,
    'do_sample':False,
}
logging.info(f"using textgen params:\n\n:{settings}")
session_settings['num_beams'] = number_beams
session_settings['length_penalty'] = length_penalty
session_settings['max_len_ratio'] = max_len_ratio

def summarize(ids, mask, **kwargs):

    ids = ids[None, :]
    mask = mask[None, :]

    input_ids = ids.to('cuda')
    attention_mask = mask.to('cuda')
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1

    summary_pred_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
            return_dict_in_generate=True,
            **kwargs
        )
    summary = tokenizer.batch_decode(
                summary_pred_ids.sequences,
                skip_special_tokens=True,
                remove_invalid_values=True,
            )

    return summary

def summarize_via_tokenbatches(
        input_text:str,
        batch_length=8192,
        batch_stride=16,
        **kwargs,):
    encoded_input = tokenizer(
                        input_text,
                        padding='max_length',
                        truncation=True,
                        max_length=batch_length,
                        stride=batch_stride,
                        return_overflowing_tokens=True,
                        add_special_tokens =False,
                        return_tensors='pt',
                    )

    in_id_arr, att_arr = encoded_input.input_ids, encoded_input.attention_mask
    gen_summaries = []

    pbar = tqdm(total=len(in_id_arr))

    for _id, _mask in zip(in_id_arr, att_arr):

        result = summarize(
            ids=_id,
            mask=_mask,
            **kwargs,
        )
        _sum = {
            "input_tokens":_id,
            "summary":result,
        }
        gen_summaries.append(_sum)
        pbar.update()

    pbar.close()

    return gen_summaries

directory = '/content/'
text_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.txt')]


for file_path in text_files:
  with open(file_path, 'r', errors='ignore') as f:
    raw_text = f.read()

  long_text = clean(raw_text)
  logging.info(f"removed {len(long_text) - len(raw_text)} chars via cleaning")
  batch = {}
  batch['text'] = long_text


  encoded_input = tokenizer(
    long_text,
    padding='max_length',
    truncation=True,
    max_length=token_batch_length,
    stride=batch_stride,
    return_overflowing_tokens=True,
    add_special_tokens =False,
    return_tensors='pt',)

  _summaries = summarize_via_tokenbatches(
    long_text,
    batch_length=token_batch_length,
    batch_stride=batch_stride,
    **settings,)

  sum_text = [s["summary"][0] for s in _summaries]
  full_summary = "\n\t".join(sum_text)
  path = Path(file_path)
  _outpath = f"SUM_{path.name}"

  with open(_outpath,"w",) as fo:
    fo.writelines(full_summary)
    fo.write("\n" * 3)
    fo.write("\n\n---\n")

  if torch.cuda.is_available():
    torch.cuda.empty_cache()

num_workers = 4

with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
    executor.map(summarize, text_files)

!cat $_outpath

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In this paper, the authors summarize the results from the spectroscopic follow-up of the first phase of the mobster mission and report the discovery of three new magnetic b stars. Of the four targets from the mobster, one--hd 25709--has a detectable surface magnetic field and is classified as an "oba", while the others have lower detection limits. They also report the discovery of a previously unknown binary star, hd 25709, and provide criteria for the selection of larger sample of stars to be followed up later on using high-resolution spectrophotometry. The authors estimate that there are currently only 11 known magnetic b stars and less than 100 known magnetic o-type stars, and with the cost of telescope equipment, it is not possible to survey large swatches of the sky with sufficient resolution to determine their true nature. Instead, they recommend that indirect methods such as photometry be used to identify candidates for further study. In this paper, they describe the approach th

**Combine Summaries for Given Author**

To understand the expertise of the author, we need to include all possible publications, so that we see an overarching view of their work.

In [16]:
datf['Summaries'] = ""

for index,row in datf.iterrows():
  identifiers = row['arxiv-id'].split(',')

  concat_summ = ""

  for identifier in identifiers:
    if identifier.strip() == 'None':
      concat_summ = 'None'
      break

    sanitized_identifier = identifier.strip().replace('/', '_')

    filename = f'SUM_{sanitized_identifier}.txt'

    if os.path.exists(filename):
      with open(filename, 'r', encoding = 'utf-8') as file:
        concat_summ += file.read()
    else:
      concat_summ = 'File not found'

  datf.at[index, 'Summaries'] = concat_summ

In [17]:
datf.reset_index(inplace = True)

**N_grams**

In [18]:
def sum_n_grams(df, directorypath): #directory path should lead to TextAnalysis.py
    top10Dict = {'Summaries Top 10 Words':[],
                 'Summaries Top 10 Bigrams':[],
                 'Summaries Top 10 Trigrams':[]}

    for i in df.values:
        summaries = i[15]

        top10words = TA.topwords(summaries, directorypath)
        top10bigrams = TA.topbigrams(summaries, directorypath)
        top10trigrams = TA.toptrigrams(summaries, directorypath)

        top10Dict['Summaries Top 10 Words'].append(top10words)
        top10Dict['Summaries Top 10 Bigrams'].append(top10bigrams)
        top10Dict['Summaries Top 10 Trigrams'].append(top10trigrams)

    sum_top10Df = df
    sum_top10Df['Summaries Top 10 Words'] = top10Dict['Summaries Top 10 Words']
    sum_top10Df['Summaries Top 10 Bigrams'] = top10Dict['Summaries Top 10 Bigrams']
    sum_top10Df['Summaries Top 10 Trigrams'] = top10Dict['Summaries Top 10 Trigrams']

    sum_top10Df = sum_top10Df[['Input Author', 'Input Institution', 'First Author', 'Bibcode', 'Title', 'Publication Date',
             'Keywords', 'Affiliations', 'Abstract', 'Top 10 Words', 'Top 10 Bigrams', 'Top 10 Trigrams', 'Data Type',
            'arxiv-id', 'Summaries', 'Summaries Top 10 Words', 'Summaries Top 10 Bigrams', 'Summaries Top 10 Trigrams']]

    return sum_top10Df

In [19]:
path_stop= '/content/'
stop_file='stopwords.txt'
stop_dir=path_stop+stop_file

final_df = sum_n_grams(datf, stop_dir)
#final_df

**Save New File**

Change the name from results to preferred naming convention.

In [22]:
final_df.to_csv('Results.csv', index = False)